<hr>

<table style="width:100%">
  <tr>
    <th><img align="center" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/UNAL_Aplicación_Medell%C3%ADn.svg/1280px-UNAL_Aplicación_Medell%C3%ADn.svg.png" width="300"/></th>
    <th><img align="center" src="http://www.redttu.edu.co/es/wp-content/uploads/2016/01/itm.png" width="300"/> </th> 
    <th><img align="center" src="https://www.cienciasdelaadministracion.uns.edu.ar/wp-content/themes/enlighten-pro/images/logo-uns-horizontal.png" width="300"/></th>
  </tr>
</table>


<hr>

#### Pedro Atencio Ortiz - 2019 (pedroatencio@itm.edu.co)

# Módulo 3. Conceptos utilitarios

En este notebook abordaremos los siguientes tópicos:

1. Callbacks: Tomar decisiones durante el proceso de entrenamiento. 
2. Lamba layers: Construir nuestras propias capas de red neuronal.
3. Estimación del $learning\_rate$. 
4. Grid search: Encontrar los mejores parámetros de la red.
5. Custom losses: Construir nuestras propias funciones de error.
6. Custom Activations: Construir nuestras propias funciones de activación.

In [ ]:
# Funciones utilitarias

import numpy as np
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

def generate_data(data_type, noise=0.2, num_samples=200):
    
    np.random.seed(0)
    if data_type == 'moons':
        X, Y = datasets.make_moons(num_samples, noise=noise)
    elif data_type == 'circles':
        X, Y = sklearn.datasets.make_circles(num_samples, noise=noise)
    elif data_type == 'blobs':
        X, Y = sklearn.datasets.make_blobs(centers=2, cluster_std=noise)
    return X, Y

def visualize_model(model, X, Y, output='truncate', save=False, save_path=None):
    XT = np.copy(X)
    # Set min and max values and give it some padding
    x_min, x_max = XT[:, 0].min() - .5, XT[:, 0].max() + .5
    y_min, y_max = XT[:, 1].min() - .5, XT[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    if(output=='truncate'):
        Z = np.round(model.predict(np.c_[xx.ravel(), yy.ravel()]))
    elif(output=='same'):
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    else:
        print("output param must be either truncate or same")
        return False

    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.figure(figsize=(7,5))
    plt.contourf(xx, yy, Z, cmap=plt.cm.bone)

    color = ['blue' if y == 1 else 'red' for y in np.squeeze(Y)]
    plt.scatter(X[:,0], X[:,1], color=color)

    if(save):
        plt.savefig(save_path)
    else:
        plt.show()

<hr>

## 1. Callbacks

Los callbacks son funciones que nos permiten realizar tareas personalizadas durante el entrenamiento de la red neuronal. <a href="https://keras.io/callbacks/">Keras </a> tiene un conjunto de callbacks predefinidos, y permite utilizar la clase abstracta __Callback__ para construidas callbacks personalizadas.

Entre los callbacks personalizados más utilizados se encuentran:

- EarlyStopping
- LearningRateScheduler
- TerminateOnNaN

Probemos algunos de estos callbacks.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, LeakyReLU, Activation
from tensorflow.keras.optimizers import RMSprop, Adam

import matplotlib.pyplot as plt

In [ ]:
X, Y = generate_data('moons', 0.3)

m = len(X)

indices = np.arange(m) #creamos los indices ordenados del dataset
indices_permutados = np.random.permutation(indices)

train_fraction = 0.8
train_index = indices_permutados[:int(m*train_fraction)]
test_index = indices_permutados[int(m*train_fraction):]

color = ['blue' if y == 1 else 'red' for y in np.squeeze(Y)] # una lista para darle color a las clases

plt.figure(figsize=(7,5))
plt.scatter(X[:,0], X[:,1], color=color)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid()

plt.show()

Utilicemos las capas tipo __Dense__ con activaciones __elu__ en las capas intermedias, y __softmax__ para la salida. También utilicemos la capa __Flatten__ para aplanar la imagen de entrada a un vector.

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

# Red neuronal
model = Sequential()
model.add(Dense(units=10, use_bias=True, activation='elu', input_dim=2))
model.add(Dense(units=1, activation='sigmoid', use_bias=True))

opt = RMSprop(lr=0.1)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])

print(model.summary())

Ejecutaremos el ciclo de entrenamiento con tres callbacks de la siguiente forma:

- __EarlyStopping (callbk1)__: Haremos un monitoreo sobre el error de validación y si crece en muchas ocasiones, detenemos el entrenamiento.
- __LearningRateScheduler (callbk2)__: Cada 100 epocas actualizamos el learning rate al 90% del valor actual.
- __CustomCallback (callbk3)__: Cada 20 epocas guardaremos una imagen de la clasificación del modelo que luego podemos utilizar para visualizar la dinamica del entrenamiento.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, Callback

In [ ]:
from math import exp

def schedule(epoch, lr):
    
    # Cada 100 epocas actualice el learning rate al 0.9 del actual
    
    if((epoch+1) % 100 == 0):
        lr = lr * 0.9
        print("Learning rate update: %.4f \n"%(lr))

    return lr

class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if ((epoch) % 20 == 0):
            visualize_model(model, X, Y, output='same', 
                            save=True, 
                            save_path='img'+str(epoch+1)+'.png')

In [ ]:
callbk1 = EarlyStopping(monitor='val_loss', 
                       mode='min',
                       patience=50,
                       restore_best_weights=True,
                       verbose=True)

callbk2 = LearningRateScheduler(schedule)

callbk3 = CustomCallback()

history = model.fit(X[train_index], Y[train_index],
                    validation_data=[X[test_index], Y[test_index]],
                    epochs=1000, verbose=2,
                    callbacks=[callbk1, callbk2, callbk3])

In [ ]:
print("Error (Loss) final: %.4E"%(np.array(history.history['loss'][-1:]))) #Error final de la lista de errores
print("Precision (Accuracy) final: %.4f"%(np.array(history.history['acc'][-1:])))

plt.figure(figsize=(10,3))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['acc'])

plt.legend(['train_loss', 'val_loss', 'train_acc', 'val_acc'])

plt.show()

In [ ]:
visualize_model(model, X, Y, output='truncate')

### Trabajemos

Cree un Callback para que termine el entrenamiento si la diferencia entre el error de entrenamiento y el de validación supera un umbral dado.

Para ello debe utilizar, el diccionario __logs__ dentro de la función __on_epoch_end__ y __self.model.stop_training = True__.

<hr>

## 2. Lambda layers

Las capas lambda permiten crear capas personalizadas que ejecuten una función Lambda. Por ejemplo, promediar los valores de una capa anterior, realizar una operación aritmética sobre una capa, concatenar la activación de dos o más capas, etc.

In [ ]:
#funciones / expresiones Lambda

f = lambda x: x**0.5
g = lambda a,b: a**2 + 2*a*b + b**2

print(f(2))
print(g(2,4))

fact = lambda x: 1 if x == 0 else x * fact(x-1)
print("Factorial de %i: %i"%(4,fact(4)))

fib = lambda i: i if i < 2 else fib(i-1)+fib(i-2)
print("%i-esimo elemento Fibonnaci: %i"%(20, fib(20)))

Ahora crearemos un modelo con una capa Lambda

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

# Red neuronal
model = Sequential()
model.add(Dense(units=10, use_bias=True, activation='linear', input_dim=2))

model.add(Lambda(lambda x: (K.exp(x) - K.exp(-x)) / (K.exp(x) + K.exp(-x)), 
                 name='tanh'))

model.add(Dense(units=1, activation='sigmoid', use_bias=True))

opt = RMSprop(lr=0.1)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])

print(model.summary())

In [ ]:
history = model.fit(X[train_index], Y[train_index],
                    validation_data=[X[test_index], Y[test_index]],
                    epochs=300, verbose=0)

In [ ]:
visualize_model(model, X, Y, output='truncate')

In [ ]:
def relu(x):
    return np.maximum(x, 0)

x = np.linspace(-10,10,50)
plt.plot(x,relu(x))

<hr>

## Trabajemos

Utilizando como base la función relu implementada sobre numpy, implemente una capa Lambda que ejecute la función relu.

<hr>

Las capas Lambda no se limitan a trabajar con expresiones Lambda. Podemos hacer un llamado a una función externa y utilizar el cableado tipo C.

In [ ]:
def layers_sum(vects):
    x, y = vects
    return x+y

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

x1 = Input(shape=(1,))
x2 = Input(shape=(2,))

a1 = Dense(units=10, activation='relu')(x1)
a1 = Dense(units=10, activation='relu')(a1)

a2 = Dense(units=10, activation='relu')(x2)

a3 = Lambda(layers_sum, output_shape=(10,))([a1,a2])
output = Dense(units=1, activation='sigmoid')(a3)

model = Model(inputs=[x1,x2], outputs=output)

In [ ]:
from tensorflow.keras.utils import plot_model 

In [ ]:
plot_model(model)

<hr>

## 3. Estimación del learning rate $\alpha$

Una forma de estimar este meta-parámetro, consiste en entrenar el modelo haciendo un barrido de $\alpha$ mediante CallBacks, plotear el comportamiento del error, y seleccionar el valor límite de $\alpha$ donde el error se mantiene estable.

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

initial_lr = 1e-4

# Red neuronal
model = Sequential()
model.add(Dense(units=10, use_bias=True, activation='elu', input_dim=2))
model.add(Dense(units=1, activation='sigmoid', use_bias=True))

opt = RMSprop(lr=initial_lr)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])

print(model.summary())

In [ ]:
num_epochs = 75

lr_schedule = LearningRateScheduler(lambda epoch: initial_lr * 10**(epoch / 20))

history = model.fit(X[train_index], Y[train_index], 
                    epochs=num_epochs, 
                    callbacks=[lr_schedule], verbose=0)

In [ ]:
max_lr = initial_lr * 10**(num_epochs / 20)

plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([initial_lr, max_lr, 0, np.max(history.history["loss"])])

plt.xlabel("learning rate")
plt.ylabel("loss")

plt.show()

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

lr = 3e-2

# Red neuronal
model = Sequential()
model.add(Dense(units=10, use_bias=True, activation='elu', input_dim=2))
model.add(Dense(units=1, activation='sigmoid', use_bias=True))

opt = RMSprop(lr=lr)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])

print(model.summary())

In [ ]:
num_epochs = 500

history = model.fit(X[train_index], Y[train_index], 
                    validation_data=[X[test_index], Y[test_index]],
                    epochs=num_epochs, verbose=0)

In [ ]:
visualize_model(model, X, Y, output='same')

In [ ]:
print("Error (Loss) final: %.4E"%(np.array(history.history['loss'][-1:]))) #Error final de la lista de errores
print("Precision (Accuracy) final: %.4f"%(np.array(history.history['acc'][-1:])))

plt.figure(figsize=(10,3))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['acc'])

plt.legend(['train_loss', 'val_loss', 'train_acc', 'val_acc'])

plt.show()

<hr>

## 4. Grid Search

Grid-search consiste en proceso de búsqueda de meta-parámetros asociados a un modelo de predicción. Para esto utilizaremos un wrapper de Keras$\rightarrow$scikit-learn con el objetivo de utilizar el grid-search de sklearn sobre un modelo de Keras.

In [ ]:
#Importamos el wrapper de Keras->sklearn

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#Primero creamos una funcion que crea la red neuronal y--
#--pasamos a esta funcion los parametros asociados al modelo que desean ser explorados.

def crear_modelo(lr=3e-2, n_units=2):
    
    # Red neuronal
    model = Sequential()
    model.add(Dense(units=n_units, use_bias=True, activation='elu', input_dim=2))
    model.add(Dense(units=1, activation='sigmoid', use_bias=True))

    opt = RMSprop(lr=lr)

    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    
    return model

In [ ]:
#Ahora creamos un modelo KerasClassifier.
#El constructor de este wrapper puede acceder a los parametros--
#--de la funcion de creacion y a los de model.fit().

keras_model = KerasClassifier(build_fn=crear_modelo, n_units=5)

In [ ]:
#Importamos el grid-search de sklearn

from sklearn.model_selection import GridSearchCV

In [ ]:
#creamos el diccionario de parametros a ser explorados

param_grid = dict(n_units=[5,10,15], 
                  epochs=[350,400], 
                  batch_size=[8,16])

#cv=cross-validation
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# Imprimir resultado

print("Mejor acc(val. cruz.): %.3f utilizando %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%.3f (%.4f) con: %r" % (mean, stdev, param))

<hr>

## 5. Custom losses (errores personalizados)

En algunas ocasiones es posible necesitar funciones de error personalizadas las cuales no pueden construidas directamente a partir de las funciones de error de Keras. 

In [ ]:
#Creamos una funcion que recibe los parametros __y_true__ y __y_pred__.

def error_func(y_true, y_pred):
    '''
    Log Loss
    '''
    loss = -(y_true*K.log(y_pred) + (K.constant(1.0)-y_true)*K.log(K.constant(1.0)-y_pred))
    
    cost = K.mean(loss)
    
    return cost

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

lr = 3e-2

# Red neuronal
model = Sequential()
model.add(Dense(units=10, use_bias=True, activation='elu', input_dim=2))
model.add(Dense(units=1, activation='sigmoid', use_bias=True))

opt = RMSprop(lr=lr)

model.compile(optimizer=opt,
              loss=error_func, #utilizamos nuestra funcion de error
              metrics=['acc'])

print(model.summary())

In [ ]:
history = model.fit(X[train_index], Y[train_index],
                    validation_data=[X[test_index], Y[test_index]],
                    epochs=300, verbose=0)

In [ ]:
visualize_model(model, X, Y, output='same')

<hr>

## Analicemos

Debido a la restriccion en Keras de que las loss functions reciben exclusivamente los parametros y_pred y y_true, en caso de necesitar funciones de error que reciben distintos o mayor numero de parametros, podemos utilizar funcion clojure:
<br>
https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618

In [ ]:
#Ejemplo de clojure

def funcion1(c):
    
    def funcion2(a,b):
        return(a+b+c)
    
    return funcion2

In [ ]:
f = funcion1(5)

<hr>

## 6. Custom Activations

En algunas ocasiones es posible necesitar funciones de activación personalizadas las cuales no se encuentran implementadas en el motor de Keras. 

Utilicemos como ejemplo la implementación de la función Mish entregada por los autores.

<br>

<center>
<font size=3>
    $a = z \cdot tanh(\zeta (z))$
    <br>
    where
    <br>
    $\zeta(z) = ln(1+e^z)$
</font>
</center>

https://arxiv.org/pdf/1908.08681.pdf

In [ ]:
z = np.linspace(-5,5,50)
a = z * np.tanh(np.log(1+np.exp(z)))

plt.plot(z,a)
plt.grid()

In [ ]:
from tensorflow.keras.utils import get_custom_objects

class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    #return inputs * tf.math.tanh(tf.math.softplus(inputs))
    return inputs * tf.math.tanh(tf.log(1+tf.exp(inputs)))

#actualizamos el diccionario global del objetos propios
get_custom_objects().update({'Mish': Mish(mish)}) 

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

lr = 3e-2

# Red neuronal
model = Sequential()
model.add(Dense(units=10, use_bias=True, input_dim=2))
model.add(Activation(activation='Mish'))
model.add(Dense(units=1, activation='sigmoid', use_bias=True))

opt = RMSprop(lr=lr)

model.compile(optimizer=opt,
              loss='binary_crossentropy', #utilizamos nuestra funcion de error
              metrics=['acc'])

print(model.summary())

In [ ]:
history = model.fit(X[train_index], Y[train_index],
                    validation_data=[X[test_index], Y[test_index]],
                    epochs=500, verbose=0)

In [ ]:
visualize_model(model, X[test_index], Y[test_index], output='truncate')